In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import os

# Obtener ruta absoluta 
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

plt.style.use("seaborn-v0_8-whitegrid")
from src.data.clean_columns import clean_dataframe_columns
from src.utils.constants import(
    VARS_BINARIAS,
    VARS_CATEGORICAS_NOMINALES,
    VARS_CATEGORICAS_ORDINALES,
    VARS_NUMERICAS,
    TARGET,
    TARGET_VALUES,
    LABELS    
)

In [3]:
# Cargar dataset
df = pd.read_csv('../data/raw/data.csv', delimiter=';')
df = clean_dataframe_columns(df)

print("Dataset cargado correctamente\n")
df.head()

Dataset cargado correctamente



,marital_status,application_mode,application_order,course,daytimeevening_attendance,previous_qualification,previous_qualification_grade,nacionality,mothers_qualification,fathers_qualification,...,curricular_units_2nd_sem_credited,curricular_units_2nd_sem_enrolled,curricular_units_2nd_sem_evaluations,curricular_units_2nd_sem_approved,curricular_units_2nd_sem_grade,curricular_units_2nd_sem_without_evaluations,unemployment_rate,inflation_rate,gdp,target
0,1,17,5,171,1,1,122.0,1,19,12,...,0,0,0,0,0.000000,0,10.8,1.4,1.74,Dropout
1,1,15,1,9254,1,1,160.0,1,1,3,...,0,6,6,6,13.666667,0,13.9,-0.3,0.79,Graduate
2,1,1,5,9070,1,1,122.0,1,37,37,...,0,6,0,0,0.000000,0,10.8,1.4,1.74,Dropout
3,1,17,2,9773,1,1,122.0,1,38,37,...,0,6,10,5,12.400000,0,9.4,-0.8,-3.12,Graduate
4,2,39,1,8014,0,1,100.0,1,37,38,...,0,6,6,6,13.000000,0,13.9,-0.3,0.79,Graduate


In [4]:
print("==============================================================")
print("1. DIMENSIÓN DEL DATASET")
print("==============================================================")
print(f"Filas: {df.shape[0]}")
print(f"Columnas: {df.shape[1]}")
print(f"Celdas totales: {df.shape[0] * df.shape[1]:,}")

1. DIMENSIÓN DEL DATASET
Filas: 4424
Columnas: 37
Celdas totales: 163,688


In [5]:
print("==============================================================")
print("2. COMPLETITUD - Valores Nulos")
print("==============================================================")

nulls = df.isnull().sum()
nulls_pct = (nulls / len(df)) * 100

tabla_nulos = pd.DataFrame({
    "nulos": nulls,
    "% nulos": nulls_pct.round(2)
})

display(tabla_nulos)

print("\nTotal de valores nulos en dataset:", nulls.sum())

2. COMPLETITUD - Valores Nulos


,nulos,% nulos
marital_status,0,0.0
application_mode,0,0.0
application_order,0,0.0
course,0,0.0
daytimeevening_attendance,0,0.0
previous_qualification,0,0.0
previous_qualification_grade,0,0.0
nacionality,0,0.0
mothers_qualification,0,0.0
fathers_qualification,0,0.0



Total de valores nulos en dataset: 0


In [6]:
print("==============================================================")
print("3. CONSISTENCIA - Duplicados")
print("==============================================================")

duplicates = df.duplicated().sum()
pct_dup = duplicates / len(df) * 100

print(f"Porcentaje de filas duplicadas: {pct_dup:.2f}%")

print(f"\nRegistros duplicados: {duplicates}")

if duplicates > 0:
    display(df[df.duplicated()])

3. CONSISTENCIA - Duplicados
Porcentaje de filas duplicadas: 0.00%

Registros duplicados: 0


In [7]:
print("==============================================================")
print("3. CONSISTENCIA - Distribuciones Sesgadas")
print("==============================================================")

"""
    Calcula un puntaje de calidad basado en la distribución estadística de una variable numérica, utilizando medidas de asimetría (skewness) y curtosis (kurtosis).

    El objetivo del puntaje es evaluar qué tan "normal" o equilibrada es la distribución de la variable. Las distribuciones altamente sesgadas o con colas pesadas reciben penalizaciones, dado que pueden afectar 
    negativamente los modelos de machine learning.

    Parámetros
    ----------
    col : str
        Nombre de la columna del DataFrame (df) a evaluar.

    Lógica de Cálculo
    -----------------
    1. Se calcula la asimetría absoluta (skewness):
         - Penalización: skew * 10
         - Penalización máxima a aplicar 40 puntos

    2. Se calcula la curtosis absoluta (kurtosis):
         - Penalización: kurt * 5
         - Penalización máxima a aplica 40 puntos

    3. Se parte de un puntaje base de 100 y se restan las penalizaciones.

    4. El puntaje final se trunca en 0 para evitar valores negativos.

    Retorna
    -------
    int
        Puntaje de calidad entre 0 y 100, donde:
            - 100 indica distribución ideal (similar a normal)
            - 70-90 indica distribución aceptable
            - 40-70 sugiere problemas de asimetría o colas pesadas
            - < 40 indica distribución crítica y posiblemente distorsionada
    """


def score_distribucion(col):

    
    skew = abs(df[col].skew())
    kurt = abs(df[col].kurtosis())

    score = 100

    score -= min(skew * 10, 40)     # penalización por skew
    score -= min(kurt * 5, 40)      # penalización por curtosis

    return max(score, 0)

3. CONSISTENCIA - Distribuciones Sesgadas


In [8]:
print("==============================================================")
print("3. CONSISTENCIA - Categorias no representativas")
print("==============================================================")

"""
    Calcula un puntaje de calidad para variables categóricas basado
    en la cantidad de categorías con baja frecuencia ("categorías raras").

    Las categorías raras pueden generar inestabilidad en modelos predictivos,
    especialmente cuando se aplica one-hot encoding o técnicas basadas en 
    frecuencias. Este puntaje permite evaluar el riesgo asociado a la baja 
    representatividad de ciertas categorías.

    Parámetros
    ----------
    col : str
        Nombre de la columna del DataFrame a evaluar.

    umbral : int, opcional (default=10)
        Número mínimo de observaciones requerido para que una categoría
        sea considerada suficientemente representativa.
        Las categorías con frecuencia < umbral son clasificadas como "raras".

    Lógica de Cálculo
    -----------------
    1. Se contabiliza cuántas categorías tienen menos de `umbral` observaciones.
    2. Se asigna un puntaje según la severidad del problema:

        - 0 categorías raras     → Score = 100 (excelente)
        - 1-2 categorías raras   → Score = 80 (riesgo bajo)
        - 3-5 categorías raras   → Score = 60 (riesgo moderado)
        - > 5 categorías raras    → Score = 30 (riesgo alto)

    Retorna
    -------
    int
        Puntaje de calidad entre 30 y 100:
            - 100 indica que no existen categorías raras
            - 80 indica leve riesgo
            - 60 indica riesgo moderado por poca representatividad
            - 30 indica riesgo severo para modelado
    """

def score_categorias_raras(col, umbral=10):
    conteos = df[col].value_counts()

    n_raras = (conteos < umbral).sum()

    if n_raras == 0:
        return 100
    elif n_raras <= 2:
        return 80
    elif n_raras <= 5:
        return 60
    else:
        return 30

3. CONSISTENCIA - Categorias no representativas


In [9]:
print("==============================================================")
print("4. EXACTITUD - Validación de categorías fuera de dominio")
print("==============================================================")

errores_dominio = {}

for col, mapping in LABELS.items():
    valores_validos = set(mapping.keys())
    valores_actuales = set(df[col].unique())
    fuera = valores_actuales - valores_validos

    if len(fuera) > 0:
        errores_dominio[col] = list(fuera)
errores_dominio

# Calcula score 
score_dominio = {}
for col in df.columns:
    if col not in errores_dominio:
        score_dominio[col] = 100
    else:
        n = len(errores_dominio[col])
        score_dominio[col] = max(0, 100 - 20 * n)


if len(errores_dominio) == 0:
    print("\nNo existen valores fuera del dominio definido.")
else:
    print(" Valores fuera de dominio encontrados:")

    rows = []
    for variable, valores in errores_dominio.items():
        for v in valores:
            rows.append({"Variable": variable, "Valor fuera de dominio": v})

    df_errores = pd.DataFrame(rows)
    display(df_errores)

4. EXACTITUD - Validación de categorías fuera de dominio

No existen valores fuera del dominio definido.


In [10]:
print("==============================================================")
print("6. OUTLIERS - Método IQR")
print("==============================================================")

outlier_report = []

numericas = df.select_dtypes(include=["int64", "float64"]).columns

for col in numericas:

    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1

    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR

    n_outliers = ((df[col] < lower) | (df[col] > upper)).sum()
    pct = round(n_outliers / len(df) * 100, 2)

    outlier_report.append([col, n_outliers, pct])

tabla_outliers = pd.DataFrame(outlier_report,columns=["Variable", "Outliers IQR", "% IQR"])

# Calcula score de rangos
score_rangos = {}
for _, row in tabla_outliers.iterrows():
    score_rangos[row["Variable"]] = max(0, 100 - row["% IQR"] * 1.2)

# Las no-numéricas reciben 100
for col in df.columns:
    score_rangos.setdefault(col, 100)

display(tabla_outliers)

6. OUTLIERS - Método IQR


,Variable,Outliers IQR,% IQR
0,marital_status,505,11.42
1,application_mode,0,0.00
2,application_order,541,12.23
3,course,442,9.99
4,daytimeevening_attendance,483,10.92
5,previous_qualification,707,15.98
6,previous_qualification_grade,179,4.05
7,nacionality,110,2.49
8,mothers_qualification,0,0.00
9,fathers_qualification,0,0.00


In [11]:
print("==============================================================")
print("7. EXACTITUD – Validación de tipos de datos")
print("==============================================================")

df.dtypes


7. EXACTITUD – Validación de tipos de datos


marital_status                                    int64
application_mode                                  int64
application_order                                 int64
course                                            int64
daytimeevening_attendance                         int64
previous_qualification                            int64
previous_qualification_grade                    float64
nacionality                                       int64
mothers_qualification                             int64
fathers_qualification                             int64
mothers_occupation                                int64
fathers_occupation                                int64
admission_grade                                 float64
displaced                                         int64
educational_special_needs                         int64
debtor                                            int64
tuition_fees_up_to_date                           int64
gender                                          

In [12]:
print("==============================================================")
print("8. ÍNDICE GLOBAL DE CALIDAD DEL DATASET")
print("==============================================================")

# Crear base del índice con todas las columnas del dataset
metricas = pd.DataFrame(index=df.columns)

#### Dimensión COMPLETITUD ####
# --- 1. % NULOS ---
metricas["Valores_nulos"] = 100 - nulls_pct


#### Dimensión CONSISTENCIA ####
# --- 2. Duplicados ---
metricas["Duplicados"] = 100 - pct_dup

# --- 3. % Distribuciones Sesgadas ---
metricas["Score_sesgo"] = [
    score_distribucion(col) if col in VARS_NUMERICAS else np.nan
    for col in df.columns
]
# --- 4. % Categorias no representativas ---
metricas["Score_ategorías_raras"] = [
    score_categorias_raras(col) if col in VARS_CATEGORICAS_NOMINALES else np.nan
    for col in df.columns
]

#### Dimensión EXACTITUD ####
# --- 3. EXACTITUD ---
metricas["Exactitud_dominio"] = metricas.index.map(score_dominio)
metricas["Exactitud_rangos"] = metricas.index.map(score_rangos).round(2)

metricas["Score Calidad"] = (
    0.1666 * metricas["Valores_nulos"] +
    0.1666 * metricas["Duplicados"] +
    0.1666 * metricas["Score_sesgo"].fillna(100) +
    0.1666 * metricas["Score_ategorías_raras"].fillna(100) + 
    0.1666 * metricas["Exactitud_dominio"] +
    0.1666 * metricas["Exactitud_rangos"] 
).round(2)

metricas = metricas.round({
    "Valores_nulos": 2,
    "Duplicados": 2,
    "Score_sesgo": 2,
    "Score_ategorías_raras": 2,
    "Exactitud_dominio": 2,
    "Exactitud_rangos": 2, 
})

def clasificar(score):
    if score >= 90:
        return "Excelente"
    elif score >= 80:
        return "Muy Buena"
    elif score >= 70:
        return "Aceptable"
    elif score >= 60:
        return "Baja"
    else:
        return "Crítica"

metricas["Nivel Calidad"] = metricas["Score Calidad"].apply(clasificar)
metricas_sorted = metricas.sort_values("Score Calidad")
metricas_sorted = metricas_sorted.sort_values("Score Calidad")
metricas_sorted.to_csv("../outputs/tables/calidad_datos/indice_calidad_dataset.csv", index=True)
print("Tabla de índice de calidad guardada en outputs/tables/calidad_datos")

display(metricas.sort_values("Score Calidad"))

8. ÍNDICE GLOBAL DE CALIDAD DEL DATASET
Tabla de índice de calidad guardada en outputs/tables/calidad_datos


,Valores_nulos,Duplicados,Score_sesgo,Score_ategorías_raras,Exactitud_dominio,Exactitud_rangos,Score Calidad,Nivel Calidad
curricular_units_1st_sem_credited,100.0,100.0,20.00,NaN,100,84.35,84.02,Muy Buena
curricular_units_2nd_sem_credited,100.0,100.0,20.00,NaN,100,85.62,84.24,Muy Buena
previous_qualification,100.0,100.0,NaN,30.0,100,80.82,85.10,Muy Buena
curricular_units_1st_sem_without_evaluations,100.0,100.0,20.00,NaN,100,92.02,85.30,Muy Buena
curricular_units_2nd_sem_without_evaluations,100.0,100.0,20.00,NaN,100,92.36,85.36,Muy Buena
mothers_occupation,100.0,100.0,NaN,30.0,100,95.07,87.48,Muy Buena
fathers_occupation,100.0,100.0,NaN,30.0,100,95.20,87.50,Muy Buena
nacionality,100.0,100.0,NaN,30.0,100,97.01,87.80,Muy Buena
fathers_qualification,100.0,100.0,NaN,30.0,100,100.00,88.30,Muy Buena
mothers_qualification,100.0,100.0,NaN,30.0,100,100.00,88.30,Muy Buena
